# Audio Decomposition

In this homework we will explore NMF-based audio decomposition.

We are getting close to the final project, so I will be providing very little hand holding in this assignment.  I will simply give you broad tasks to accomplish without providing decomposed functions or answers.  It is up to you to verify that your code is working!

Please note that part of your grade will be on the readability and decomposition of your code, so make sure your code is clean and organized.  Part of your grade will also be on the clarity of your writing.  You should explain your thought process clearly and support your claims with appropriate plots.

Team member #1: __________

Hours spent on hw: ____________

Team member #2: ___________

Hours spent on hw: ____________

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import librosa as lb
import matplotlib.pyplot as plt
import IPython.display as ipd
import scipy.io as sio

### NMF (30 points)

In this section you will implement the NMF algorithm.  You should do the following:
- Calculate the STFT of the given audio file ('xylophone.mp3').
- Implement the NMF algorithm as described in class.
- Describe a suitable stopping criterion and justify your selection with logic and/or experimental plots.
- Run NMF on the magnitude STFT
- Visualize the estimated template and activation matrices.  Comment on the results.

### Reconstruction (15 points)

In this section you will reconstruct the audio for the estimated left and right hand tracks.  You should do the following:
- Implement the simple approach described in class
- Implement the soft masking approach described in class
- Listen to the results of both reconstruction methods.  Comment.

### Initializing templates (15 points)

In this section you will initialize the templates to incorporate prior knowledge.  You should do the following:
- Create a function that initializes a template matrix given a set of midi note numbers.  The templates should have random positive values around harmonic frequencies (+/- half a semitone), and zeros elsewhere.
- Visualize the initialized template matrix using the notes {C6, D6, E6, F6, G6, A6, B6, C7}.
- Run NMF with the initialized template matrix
- Visualize the resulting template and activation matrices.  Comment.
- Sonify the left and right hand tracks.  Comment.

### Initializing Activations (10 points)

In this section you will initialize the activations to incorporate prior knowledge.  You may assume that you have already aligned the score and the audio, and have predicted the following note events at the following times.  You should do the following:
- Create a function that initializes an activation matrix given the note event information.  The entries should have random positive values around the note onsets, and zeros elsewhere.  You may assume a fixed note duration of 1 sec.
- Visualize the initialized activation matrix
- Run NMF with the initialized template and activation matrices
- Visualize the resulting template and activation matrices. Comment.
- Sonify the left and right hand tracks. Comment.

In [32]:
noteEvents = [(3, 1.23), (4, 1.23), (2, 2.19), (4, 2.19), (1, 3.16), (6, 3.16), 
              (0, 4.12), (7, 4.12), (1, 4.42), (6, 4.42), (2, 4.72), (5, 4.72),
              (3, 5.05), (4, 5.05), (2, 6.02), (4, 6.02), (1, 7.05), (6, 7.05),
              (0, 8.11), (7, 8.11)]

### Onset Models (10 points)

In this section you will add onset models to your system.  You should do the following:
- Create a function that initializes a note & onset template matrix given a set of midi note numbers.  The note templates should have random positive values around harmonic frequencies, and zeros elsewhere.  The onset templates should have all random positive values.
- Create a function that initializes a note & onset activation matrix given the note event information.  The note activations should have random positive values for the interval (t_onset, t_onset+1.0), and zeros elsewhere.  The onset activations should have random positive values for the interval (t_onset - margin, t_onset + margin), and zeros elsewhere.
- Visualize the initialized template & activation matrices.
- Run NMF with the initialized template and activation matrices.
- Visualize the resulting template and activation matrices. Comment.
- Sonify the left and right hand tracks. Comment.